# ADA Project - DataBrewers

The aim of this project is to analyze beer preference shifts across seasons and events, with a focus on understanding consumer trends in taste, aroma, palate, and appearance. By examining variation in ratings for beer style and alcohol content among US users, and comparing different states with varying weather conditions, we seek to identify seasonal trends. To deepen our understanding, we will conduct sentiment analysis to capture the tone of reviews and references to seasons, weather, or events, and correlate these insights with ratings. Our findings will provide actionable insights for professionals (brewers, marketers, etc.) by highlighting customer preferences throughout the year. Understanding these seasonal trends enables brewers to adjust product offerings to align more closely with consumer demand. As a conclusion of to our research, we could suggest the best beer that would be the perfect match for each season/festivity, in the form of a time fresco. 

## Data pre-processing 


In [4]:
import pandas as pd
import tarfile
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import re
import wordcloud
import nltk
import sys
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

The exploratory analysis is detailed in the notebook `data_cleaning.ipynb`. We decided to choose only the BeerAdvocate dataset, as it contains a sufficient number of user ratings and reviews. By analyzing the country of origin from the users, we discovered that most of the ratings were provided by users from the United States. Therefore, we restricted the dataset to include only American reviews.
 
 The complete processing and cleaning workflow is implemented in `src/scripts/preprocessing.py`. In this script, we begin by loading the raw data files for beers, reviews, users, breweries, and ratings. User location is split into country and state, and non-US users are filtered out. We then assign a US region (e.g. Northeast, Midwest) to each entry based on the user's state. Beer styles are simplified by removing country prefixes like "Belgian" or "American" and reclassified into broader categories using a style map. The script also categorizes beer based on their Alcohol By Volume (ABV) values, dividing them into three categories (low, medium, and high) using a quantile-based thresholds. We assign a season to each reviews based on the month it was published (e.g. Spring, Summer, Fall, Winter). The dataset is filtered to only include years with at least 70,000 reviews. Missing values in the rating column are handled by removing rows where ratings are missing. Finally, the cleaned versions of the datasets are saved as compressed `csv`files in the `data/cleaned/`directory. 

In [ ]:
# Run the preprocessing script
# %run src/scripts/preprocessing.py

#### Loading the data

In [2]:
reviews = pd.read_csv('data/cleaned/reviews.csv.gz', compression='gzip')
ratings = pd.read_csv('data/cleaned/ratings.csv.gz', compression='gzip')
users = pd.read_csv('data/cleaned/users.csv.gz', compression='gzip')
beers_ba = pd.read_csv('data/cleaned/beers.csv.gz', compression='gzip')
breweries_ba = pd.read_csv('data/cleaned/breweries.csv.gz', compression='gzip')

## Sentiment analysis

In this section we look at the words used in the review to try to determine the ones that are associated to positive reviews and the ones associated to negative ones.

- We define that a beer is positively graded if the rating grade strictly bigger than 4/5.
- We define that a beer is negatively graded if the rating grade strictly less than 3/5.

This will allow use to determine the aspects of a beer that are liked or disliked by reviewers. This way we will be able to understand better the consumer preferences

In [ ]:
# We import the semansentimenttic analysis script
import src.scripts.sentiment_analysis as sent_ana

# We download the stopwords to use them in the sentiment analysis
nltk.download('stopwords')

In [ ]:
# We begin by cleaning the reviews 
reviews_clean = sent_ana.clean_data(reviews)

# Separate the reviews into two groups: those with a rating higher than 4 and those with a rating lower than 3
reviews_high = reviews_clean[reviews_clean['rating'] > 4]
reviews_low = reviews_clean[reviews_clean['rating'] < 3]

# Tokenize the reviews
bigrams_high, bigrams_low = sent_ana.get_words(reviews_high, reviews_low)

# Count bigram frequencies for each group
bigram_freq_high = Counter(bigrams_high)
bigram_freq_low = Counter(bigrams_low)

To make it easier to visualise, we use Word Clouds, this way we can easily see the pair of words that are the most used, both for high and low ratings.

In [ ]:
# Generate the word clouds
wordcloud_high = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(bigram_freq_high)
wordcloud_low = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(bigram_freq_low)

# We plot word clouds for both groups
sent_ana.plot_and_save_word_clouds(wordcloud_high, wordcloud_low)

Now that we know the pair of words that are the most used in reviews with high ratings (>4/5) and low ratings (<3/5), we can see that some pair of words seem specific to well rated beers, and some seem specific to poorly rated beers.

We further investigate this to extract pair of words used only positively and pair of words used only negatively. This will allow us to determine the user preferences and potentially what makes a beer enjoyable or not.

In [ ]:
# We compare the most common bigrams in both groups and
df_comparison = sent_ana.compare_high_low(bigram_freq_high, bigram_freq_low, reviews_high, reviews_low)

# We plot the comparison
print(df_comparison)

We can easily see that some pair of words can be associated to both positive and negative reviews, but there are also words that are used only for one of the two.

We have that the following pair of words only appear in positive reviews:
- dark chocolate
- well balanced
- dark fruits
- brown sugar
- full bodied
- medium bodied

While the followinfg pair of words only appear in negative reviews: 
- finger color
- yellow color
- light bodied
- pale yellow

## Semantic analysis

In [1]:
import src.scripts.semantic_analysis as sem_ana

In [ ]:
clean_reviews = sem_ana.get_cleaned_reviews(reviews)

In [ ]:
top_20_words,word_freq = sem_ana.get_top_words(clean_reviews, 20)

In [ ]:
sem_ana.gen_wordcloud(word_freq)

In [ ]:
sem_ana.sentiment_analysis(word_freq)

In [ ]:
sem_ana.analyse_flavours(clean_reviews)